# The First Stars: Formation under X-ray Feedback -- Structure Visualization

This notebook generates all simulation results figures for my current paper.  All simulations run on stampede.tacc.utexas.edu

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

In [ ]:
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

In [ ]:
import pyGadget

Use the final snapshot of the vanilla simulation.

In [ ]:
sims = ['vanilla', 'xr_tau_J0', 'xr_tau_J1', 'xr_tau_J2', 'xr_tau_J3']
#sims = ['vanilla', 'XR_sfr_1e-3', 'XR_sfr_1e-2', 'XR_sfr_1e-1']
n0 = [306, 327, 271, 235]
n5k = [1900, 1851, 1870, 1810, 1758]
#n5k = [1900, 1794, 1701, 1700]
t0 = '_t0'
t5k = '_t5k'
vsnap, nsnap, tag = 615, n5k, t5k

In [ ]:
i=3
sim = pyGadget.sim.Simulation('stampede/'+sims[i])
snap = sim.load_snapshot(nsnap[i], track_sinks=True)

In [ ]:
snap.gas.orient_box(centering='avg')

In [ ]:
radians = np.linspace(0,np.pi,90)
imlist = []
for theta in radians:
    view = [('y',0.917768),('z', theta)]
    img = pyGadget.visualize.project(snap, 'ndensity', '5000AU', view, centering='avg', depth=2.)
    imlist.append(img[2])

In [ ]:
from IPython.html import widgets
def plot(i):
    plt.imshow(imlist[i], cmap=plt.cm.bone)

widgets.interact(plot, i=(0,89))

In [ ]:
radians[8]

# 4 Panel Simulation Comparison Plots

For each sim, picking the snapshot just prior to the formation of the first sink.

In [ ]:
simlist = []
snaplist = []
for i,simname in enumerate(sims):
    sim = pyGadget.sim.Simulation('stampede/'+sims[i])
    sim.set_coordinate_system('physical')
    snap = sim.load_snapshot(nsnap[i], track_sinks=True)
    snap.gas.orient_box(centering='avg', view='face', dens_lim=9e11)
    simlist.append(sim)
    snaplist.append(snap)

## Disk Structure

### Density structure

In [ ]:
import copy
imlist = []
sinklist = []
scale = '15000AU'

face = [[('x', 0.29518), ('z', 0.825), ('x',np.pi/2)],
        [('y', 0.494183), ('z', 1.12956), ('x',np.pi/2)],
        [('y', 0.6), ('z', 1.2), ('x',np.pi/2)],
        [('y', 0.56478), ('z', 1.69434), ('x',np.pi/2)],
        [('y', 0.917768), ('z', 0.28239), ('x',np.pi/2)]]
edge = [[('x', 0.29518), ('z', 0.825)],
        [('y', 0.494183), ('z', 1.12956)],
        [('y', 0.6), ('z', 1.2)],
        [('y', 0.56478), ('z', 1.69434)],
        [('y', 0.917768), ('z', 0.28239)]
        ]
for view in [face, edge]:
    count = 0
    for snap in snaplist:
        imlist.append(pyGadget.visualize.project(snap, 'ndensity', scale, view[count], centering='avg', 
                                                 depth=2., dens_lim=None))
        sinklist.append(copy.deepcopy(snap.sinks))
        count += 1
#    snap.close()

In [ ]:
from mpl_toolkits.axes_grid1 import ImageGrid

sim = ['J = 0', 'J = J$_{0}$', 'J = 10 J$_{0}$', 'J = 100 J$_{0}$', 'J = 1000 J$_{0}$']
bbox_props = dict(boxstyle="round", fc="k", ec="k", alpha=0.5)
ticks = [(7,8,9,10,11),(6,7,8,9,10)]

fig = plt.figure(1, (22, 8), dpi=600)
grid = ImageGrid(fig, 111, # similar to subplot(111)
                nrows_ncols = (2, 5), # creates 4x2 grid of axes
                axes_pad=0.0, # pad between axes in inch.
                cbar_mode = 'edge', cbar_location = 'right', cbar_size='7%', cbar_pad=0.0
                )
for i in range(10):
    x = imlist[i][0]
    y = imlist[i][1]
    im = imlist[i][2]
    ax = grid[i]
    img = ax.imshow(im, extent=[x.min(),x.max(),y.min(),y.max()], cmap=plt.cm.bone, origin='lower')
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    img.set_clim((6.5,10.75))
    #img.set_clim((8,12))
    
    cb = plt.colorbar(img, cax=grid.cbar_axes[i])
    #cb.set_ticks(ticks[i/4])
    cb.set_ticks((7,8,9,10))
    cb.ax.tick_params(left='on', labelsize=15, labelcolor='k')
    #if clabel[i]: 
    cb.set_label('Log Number Density \n    [cm$^{-3}$]')
    
    for sink in sinklist[i]:
        #mscale = sink.mass*6./27. + .33
        mscale = np.log(sink.mass) +1
        ax.plot(sink.x, sink.y, 'ko', ms=mscale, mew=1)
    ax.set_xlim(x.min(), x.max())
    ax.set_ylim(y.min(), y.max())

    if i > 4:
        cb.set_ticks(ticks[i/5])
        ax.text(0.5, 0.025, sim[i-5], color='w', ha='center', va='bottom', size=18, 
                transform=grid[i].transAxes, bbox=bbox_props)

plt.show()
fig.savefig('figures/structure/disks-tau.png', bbox_inches='tight', dpi=100)